### 筆記
* annotation version: tumor as kidney(DATA_0811_2)
* INPUT_SIZE = 512
* EPOCHS = 100
* BATCH_SIZE = 2 (BATCH_SIZE = 4 電腦跑不動)
* 有在Encoder中載入用ImageNet預訓練的模型參數

### 讀取函式庫、超參數設定

In [1]:
### 讀取函式庫 ###
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import time

In [2]:
import tensorflow as tf
def solve_cudnn_error():
    gpus = tf.config.experimental.list_physical_devices('GPU')
    if gpus:
        try:
            # Currently, memory growth needs to be the same across GPUs
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
            logical_gpus = tf.config.experimental.list_logical_devices('GPU')
            print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
        except RuntimeError as e:
            # Memory growth must be set before GPUs have been initialized
            print(e)

solve_cudnn_error()

1 Physical GPUs, 1 Logical GPUs


In [3]:
dataset_dir = 'DATA_0811_2\\WW600WL100'
x_train_path = 'DATA_0811_2\\WW600WL100\\train\\images'
y_train_path = 'DATA_0811_2\\WW600WL100\\train\\annotations_tumor as kidney'
checkpoints_path = 'DATA_0811_2\WW600WL100\\Model_0905_V2'
model_name = 'ResNet152V2-Unet'
start_from_latest_checkpoints = False

INPUT_SIZE = 512 # input size equal to output size
CLASS_NUM = 2 # background and roi(kidney)

# initialize the number of epochs and batch size
EPOCHS = 100
BATCH_SIZE = 2

IMAGE_ORDERING = 'channels_last' # 不要亂改這個參數

### 用生成器(generator)生成訓練圖片
使用generator的好處在於，模型在訓練(或驗證)的過程中會依據批量大小生成圖片，不需要在程式中預先準備好資料集(通常資料集都會大到無法一口氣塞入記憶體之中)，減少記憶體的使用空間。另外，使用ImageDataGenerator可以對生成的圖片實施「資料增強(Data Augmentation)」，增加模型的泛化能力。

In [4]:
### 取得資料集中所有的檔案路徑(包含CT影像和其對應標記的路徑) ###
def get_pairs_from_paths(images_path, segs_path, ignore_non_matching=False):
    """ Find all the images from the images_path directory and
        the segmentation images from the segs_path directory
        while checking integrity of data """

    ACCEPTABLE_IMAGE_FORMATS = [".jpg", ".jpeg", ".png", ".bmp"]
    ACCEPTABLE_SEGMENTATION_FORMATS = [".png", ".bmp"]

    image_files = []
    segmentation_files = {}

    for dir_entry in os.listdir(images_path):
        if os.path.isfile(os.path.join(images_path, dir_entry)) and \
                os.path.splitext(dir_entry)[1] in ACCEPTABLE_IMAGE_FORMATS:
            file_name, file_extension = os.path.splitext(dir_entry)
            image_files.append((file_name, file_extension,
                                os.path.join(images_path, dir_entry)))

    for dir_entry in os.listdir(segs_path):
        if os.path.isfile(os.path.join(segs_path, dir_entry)) and \
           os.path.splitext(dir_entry)[1] in ACCEPTABLE_SEGMENTATION_FORMATS:
            file_name, file_extension = os.path.splitext(dir_entry)
            full_dir_entry = os.path.join(segs_path, dir_entry)
            if file_name in segmentation_files:
                raise DataLoaderError("Segmentation file with filename {0}"
                                      " already exists and is ambiguous to"
                                      " resolve with path {1}."
                                      " Please remove or rename the latter."
                                      .format(file_name, full_dir_entry))

            segmentation_files[file_name] = (file_extension, full_dir_entry)

    return_value = []
    # Match the images and segmentations
    for image_file, _, image_full_path in image_files:
        if image_file in segmentation_files:
            return_value.append((image_full_path,
                                segmentation_files[image_file][1]))
        elif ignore_non_matching:
            continue
        else:
            # Error out
            raise DataLoaderError("No corresponding segmentation "
                                  "found for image {0}."
                                  .format(image_full_path))

    return return_value

### 將CT影像的陣列轉成適合模型輸入的形式(維度轉換 + 標準化) ###
def get_image_array(image_input,
                    width, height,
                    imgNorm="sub_mean", ordering='channels_first'):
    """ Load image array from input """

    if type(image_input) is np.ndarray:
        # It is already an array, use it as it is
        img = image_input
    elif isinstance(image_input, six.string_types):
        if not os.path.isfile(image_input):
            raise DataLoaderError("get_image_array: path {0} doesn't exist"
                                  .format(image_input))
        img = cv2.imread(image_input, 1)
    else:
        raise DataLoaderError("get_image_array: Can't process input type {0}"
                              .format(str(type(image_input))))

    if imgNorm == "sub_and_divide": # 除以127.5，然後減 1
        img = np.float32(cv2.resize(img, (width, height))) / 127.5 - 1
    elif imgNorm == "sub_mean": # 減去ImageNet的平均BGR
        img = cv2.resize(img, (width, height))
        img = img.astype(np.float32)
        img[:, :, 0] -= 103.939
        img[:, :, 1] -= 116.779
        img[:, :, 2] -= 123.68
        img = img[:, :, ::-1]
    elif imgNorm == "divide": # 除以255
        img = cv2.resize(img, (width, height))
        img = img.astype(np.float32)
        img = img/255.0

    if ordering == 'channels_first':
        img = np.rollaxis(img, 2, 0)
    return img

### 將標記資料的陣列轉成適合模型輸入的形式(維度轉換) ###
def get_segmentation_array(image_input, nClasses,
                           width, height, no_reshape=False):
    """ Load segmentation array from input """

    seg_labels = np.zeros((height, width, nClasses))

    if type(image_input) is np.ndarray:
        # It is already an array, use it as it is
        img = image_input
    elif isinstance(image_input, six.string_types):
        if not os.path.isfile(image_input):
            raise DataLoaderError("get_segmentation_array: "
                                  "path {0} doesn't exist".format(image_input))
        img = cv2.imread(image_input, 1)
    else:
        raise DataLoaderError("get_segmentation_array: "
                              "Can't process input type {0}"
                              .format(str(type(image_input))))

    img = cv2.resize(img, (width, height), interpolation=cv2.INTER_NEAREST)
    img = img[:, :, 0]

    for c in range(nClasses):
        seg_labels[:, :, c] = (img == c).astype(int)

    if not no_reshape:
        seg_labels = np.reshape(seg_labels, (width*height, nClasses))

    return seg_labels

import itertools
import random
from keras.preprocessing.image import ImageDataGenerator

def image_segmentation_generator(images_path, segs_path, batch_size,
                                 n_classes, input_height, input_width,
                                 output_height, output_width, do_augment=False):

    img_seg_pairs = get_pairs_from_paths(images_path, segs_path) 
        # 取得資料集中所有的檔案路徑(包含CT影像和其對應標記的路徑)
    random.shuffle(img_seg_pairs) # 打散檔案路徑
    zipped = itertools.cycle(img_seg_pairs) 
        # 將檔案路徑用循環迭代器(iterator)封裝；範例：cycle('ABCD') --> A B C D A B C D ...
#     counter = 0

    while True:
        # 如果模型訓練完一輪訓練集中所有的資料，就重新打散檔案路徑
#         if counter >= len(img_seg_pairs):
#             counter = 0
#             random.shuffle(img_seg_pairs)
#             zipped = itertools.cycle(img_seg_pairs)             
        
        X = []
        Y = []
        for _ in range(batch_size): # batch_size多大，就取得多少份資料
#             counter += 1
            
            im, seg = next(zipped) # 取得CT影像和其對應標記的路徑

            im = cv2.imread(im, 1) # 1 = cv2.IMREAD_COLOR (讀取彩色圖片)
            seg = cv2.imread(seg, 1)
            
            if do_augment:
                ### Example of transforming images and masks together. ###
                # we create two instances with the same arguments
                data_gen_args = dict(featurewise_center=False, # 範例程式碼為True，但這裡我只是要把一張圖片變成是增強後的型態
                         featurewise_std_normalization=False, # 範例程式碼為True，但這裡我只是要把一張圖片變成是增強後的型態
                         #rotation_range=30, # 範例程式碼為90
                         width_shift_range=0.1,
                         height_shift_range=0.1,
                         zoom_range=0.2)
                image_datagen = ImageDataGenerator(**data_gen_args)
                mask_datagen = ImageDataGenerator(**data_gen_args)
                # Provide the same seed and keyword arguments to the fit and flow methods
                seed = random.randint(0,10000)
                im_itr = image_datagen.flow(im.reshape(1, 512, 512, 3), batch_size=1, seed=seed)  
                seg_itr = mask_datagen.flow(seg.reshape(1, 512, 512, 3), batch_size=1, seed=seed)  
                im = next(im_itr).reshape(512, 512, 3)
                seg = next(seg_itr).reshape(512, 512, 3)
            
            # 將CT影像和其對應的標記轉換成模型輸入的形式
            X.append(get_image_array(im, input_width,
                                     input_height, imgNorm="sub_mean", ordering=IMAGE_ORDERING)) 
                # imgNorm預設為sub_mean"，但這裡我改用圖片最常實施的正規化方法(同除以255)
            Y.append(get_segmentation_array(
                seg, n_classes, output_width, output_height))

        yield np.array(X), np.array(Y)

Using TensorFlow backend.


In [5]:
# 建立訓練資料的生成器
train_gen = image_segmentation_generator(images_path = x_train_path, segs_path = y_train_path, batch_size = BATCH_SIZE, 
                                         n_classes = CLASS_NUM, input_height = INPUT_SIZE, input_width = INPUT_SIZE, 
                                         output_height = INPUT_SIZE, output_width = INPUT_SIZE, do_augment=False)

### 設定回調函式(callbacks)
keras_segmentation自定義的回調函式好像缺了什麼，直接用在這份程式碼中並不會正常運作，因此我直接用keras.callbacks.ModelCheckpoint定期儲存模型參數。

In [6]:
from keras.callbacks import ModelCheckpoint
callbacks = [ModelCheckpoint(os.path.join(checkpoints_path, model_name + '_{epoch}.h5'), 
                             save_weights_only=True, 
                             period=5)] # period=5：每5個Epoch才會儲存一次參數

### 建立模型

In [7]:
import keras
from keras.models import *
from keras.layers import *
from keras import layers

#### Eecoder - ResNet152V2

In [8]:
def identity_block(input_tensor, kernel_size, filters, stage, block, strides=1):

    filters1, filters2, filters3 = filters

    if IMAGE_ORDERING == 'channels_last':
        bn_axis = 3
    else:
        bn_axis = 1

    name_base = 'conv' + str(stage) + '_' + 'block' + block
    
    preact = BatchNormalization(axis=bn_axis, epsilon=1.001e-5,
                                       name=name_base + '_preact_bn')(input_tensor)
    preact = Activation('relu', name=name_base + '_preact_relu')(preact)
    
    shortcut = MaxPooling2D(1, strides=strides)(input_tensor) if strides > 1 else input_tensor

    x = Conv2D(filters1, (1, 1), data_format=IMAGE_ORDERING, strides=1, use_bias=False,
               name=name_base + '_1_conv')(preact)
    x = BatchNormalization(axis=bn_axis, epsilon=1.001e-5, name=name_base + '_1_bn')(x)
    x = Activation('relu', name=name_base + '_1_relu')(x)

    x = ZeroPadding2D(padding=((1, 1), (1, 1)), name=name_base + '_2_pad')(x)
    x = Conv2D(filters2, kernel_size, data_format=IMAGE_ORDERING, strides=strides,
               use_bias=False, name=name_base + '_2_conv')(x)
    x = BatchNormalization(axis=bn_axis, epsilon=1.001e-5, name=name_base + '_2_bn')(x)
    x = Activation('relu', name=name_base + '_2_relu')(x)

    x = Conv2D(filters3, (1, 1), data_format=IMAGE_ORDERING,
               name=name_base + '_3_conv')(x)

    x = Add(name=name_base + '_out')([shortcut, x])
    return x

def conv_block(input_tensor, kernel_size, filters, stage, block, strides=1):

    filters1, filters2, filters3 = filters

    if IMAGE_ORDERING == 'channels_last':
        bn_axis = 3
    else:
        bn_axis = 1
    
    name_base = 'conv' + str(stage) + '_' + 'block' + block  
        
    preact = BatchNormalization(axis=bn_axis, epsilon=1.001e-5,
                                       name=name_base + '_preact_bn')(input_tensor)
    preact = Activation('relu', name=name_base + '_preact_relu')(preact)
    
    shortcut = Conv2D(filters3, (1, 1), data_format=IMAGE_ORDERING, 
                      strides=strides, name=name_base + '_0_conv')(preact)

    x = Conv2D(filters1, (1, 1), data_format=IMAGE_ORDERING, strides=1, use_bias=False,
               name=name_base + '_1_conv')(preact)
    x = BatchNormalization(axis=bn_axis, epsilon=1.001e-5, name=name_base + '_1_bn')(x)
    x = Activation('relu', name=name_base + '_1_relu')(x)

    x = ZeroPadding2D(padding=((1, 1), (1, 1)), name=name_base + '_2_pad')(x)
    x = Conv2D(filters2, kernel_size, data_format=IMAGE_ORDERING, strides=strides,
               use_bias=False, name=name_base + '_2_conv')(x)
    x = BatchNormalization(axis=bn_axis, epsilon=1.001e-5, name=name_base + '_2_bn')(x)
    x = Activation('relu', name=name_base + '_2_relu')(x)

    x = Conv2D(filters3, (1, 1), data_format=IMAGE_ORDERING,
               name=name_base + '_3_conv')(x)

    x = Add(name=name_base + '_out')([shortcut, x])
    return x

In [9]:
input_height = input_width = INPUT_SIZE
n_classes = classes = CLASS_NUM
use_bias = True # True for ResNet and ResNetV2, False for ResNeXt
bn_axis = 3 if IMAGE_ORDERING == 'channels_last' else 1

if IMAGE_ORDERING == 'channels_first':
    img_input = Input(shape=(3, input_height, input_width))
elif IMAGE_ORDERING == 'channels_last':
    img_input = Input(shape=(input_height, input_width, 3))

### conv1 ###
x = ZeroPadding2D(padding=((3, 3), (3, 3)), name='conv1_pad')(img_input)
x = Conv2D(64, 7, strides=2, use_bias=use_bias, name='conv1_conv')(x)
f0 = x
x = ZeroPadding2D(padding=((1, 1), (1, 1)), name='pool1_pad')(x)
x = MaxPooling2D(3, strides=2, name='pool1_pool')(x)
f1 = x

### conv2 ###
x = conv_block(x, 3, [64, 64, 256], stage=2, block='1', strides=(1, 1))
x = identity_block(x, 3, [64, 64, 256], stage=2, block='2')
x = identity_block(x, 3, [64, 64, 256], stage=2, block='3', strides=2)
f2 = x
#f2 = one_side_pad(x)

### conv3 ###
x = conv_block(x, 3, [128, 128, 512], stage=3, block='1')
for i in range(6):
    x = identity_block(x, 3, [128, 128, 512], stage=3, block=str(i+2))
x = identity_block(x, 3, [128, 128, 512], stage=3, block='8', strides=2)
f3 = x

### conv4 ###
x = conv_block(x, 3, [256, 256, 1024], stage=4, block='1')
for i in range(34):
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block=str(i+2))
x = identity_block(x, 3, [256, 256, 1024], stage=4, block='36', strides=2)
f4 = x

### conv5 ###
x = conv_block(x, 3, [512, 512, 2048], stage=5, block='1')
x = identity_block(x, 3, [512, 512, 2048], stage=5, block='2')
x = identity_block(x, 3, [512, 512, 2048], stage=5, block='3', strides=1)
f5 = x

x = BatchNormalization(axis=bn_axis, epsilon=1.001e-5,name='post_bn')(x)
x = Activation('relu', name='post_relu')(x)
x = GlobalAveragePooling2D(name='avg_pool')(x)
x = Dense(1000, activation='softmax', name='probs')(x) 
    # 為了能對Eecoder載入keras.applications用imagenet預訓練的參數，因此輸出層的神經元設成1000而非標記的類別數量

#### 對Eecoder載入keras.applications用imagenet預訓練的參數

In [10]:
BASE_WEIGHTS_PATH = (
    'https://github.com/keras-team/keras-applications/'
    'releases/download/resnet/')
WEIGHTS_HASHES = {
    'resnet50': ('2cb95161c43110f7111970584f804107',
                 '4d473c1dd8becc155b73f8504c6f6626'),
    'resnet101': ('f1aeb4b969a6efcfb50fad2f0c20cfc5',
                  '88cf7a10940856eca736dc7b7e228a21'),
    'resnet152': ('100835be76be38e30d865e96f2aaae62',
                  'ee4c566cf9a93f14d82f913c2dc6dd0c'),
    'resnet50v2': ('3ef43a0b657b3be2300d5770ece849e0',
                   'fac2f116257151a9d068a22e544a4917'),
    'resnet101v2': ('6343647c601c52e1368623803854d971',
                    'c0ed64b8031c3730f411d2eb4eea35b5'),
    'resnet152v2': ('a49b44d1979771252814e80f8ec446f9',
                    'ed17cf2e0169df9d443503ef94b23b33'),
    'resnext50': ('67a5b30d522ed92f75a1f16eef299d1a',
                  '62527c363bdd9ec598bed41947b379fc'),
    'resnext101': ('34fb605428fcc7aa4d62f44404c11509',
                   '0f678c91647380debd923963594981b3')
}

model_name='resnet152v2'
file_name = model_name + '_weights_tf_dim_ordering_tf_kernels.h5'
file_hash = WEIGHTS_HASHES[model_name][0]

weights_path = keras.utils.get_file(file_name, BASE_WEIGHTS_PATH + file_name, cache_subdir='models', file_hash=file_hash)
Model(img_input, x).load_weights(weights_path)

#### Decoder (這裡我有為了讓輸出大小和輸入大小保持一致新增一個含有上取樣層的block)

In [11]:
if IMAGE_ORDERING == 'channels_first':
    MERGE_AXIS = 1
elif IMAGE_ORDERING == 'channels_last':
    MERGE_AXIS = -1

o = f4

o = (ZeroPadding2D((1, 1), data_format=IMAGE_ORDERING))(o)
o = (Conv2D(512, (3, 3), padding='valid', data_format=IMAGE_ORDERING))(o)
o = (BatchNormalization())(o)

o = (UpSampling2D((2, 2), data_format=IMAGE_ORDERING))(o)
o = (concatenate([o, f3], axis=MERGE_AXIS))
o = (ZeroPadding2D((1, 1), data_format=IMAGE_ORDERING))(o)
o = (Conv2D(256, (3, 3), padding='valid', data_format=IMAGE_ORDERING))(o)
o = (BatchNormalization())(o)

o = (UpSampling2D((2, 2), data_format=IMAGE_ORDERING))(o)
o = (concatenate([o, f2], axis=MERGE_AXIS))
o = (ZeroPadding2D((1, 1), data_format=IMAGE_ORDERING))(o)
o = (Conv2D(128, (3, 3), padding='valid', data_format=IMAGE_ORDERING))(o)
o = (BatchNormalization())(o)

o = (UpSampling2D((2, 2), data_format=IMAGE_ORDERING))(o)
o = (concatenate([o, f1], axis=MERGE_AXIS))
o = (ZeroPadding2D((1, 1), data_format=IMAGE_ORDERING))(o)
#o = (Conv2D(64, (3, 3), padding='valid', data_format=IMAGE_ORDERING))(o)
o = (Conv2D(128, (3, 3), padding='valid', data_format=IMAGE_ORDERING))(o)
o = (BatchNormalization())(o)

o = (UpSampling2D((2, 2), data_format=IMAGE_ORDERING))(o)
o = (concatenate([o, f0], axis=MERGE_AXIS))
o = (ZeroPadding2D((1, 1), data_format=IMAGE_ORDERING))(o)
#o = (Conv2D(32, (3, 3), padding='valid', data_format=IMAGE_ORDERING))(o)
o = (Conv2D(64, (3, 3), padding='valid', data_format=IMAGE_ORDERING))(o)
o = (BatchNormalization())(o)

o = (UpSampling2D((2, 2), data_format=IMAGE_ORDERING))(o)
o = (ZeroPadding2D((1, 1), data_format=IMAGE_ORDERING))(o)
#o = (Conv2D(32, (3, 3), padding='valid', data_format=IMAGE_ORDERING))(o)
o = (Conv2D(64, (3, 3), padding='valid', data_format=IMAGE_ORDERING))(o)
o = (BatchNormalization())(o)

o = Conv2D(n_classes, (3, 3), padding='same', data_format=IMAGE_ORDERING)(o) # 輸出層

## 模仿 keras_segmentation 在輸出層之後額外添加這幾層 ###
if IMAGE_ORDERING == 'channels_first':
    o = (Reshape((n_classes, INPUT_SIZE * INPUT_SIZE)))(o)
    o = (Permute((2, 1)))(o)
elif IMAGE_ORDERING == 'channels_last':
    o = (Reshape((INPUT_SIZE * INPUT_SIZE, n_classes)))(o)
o = (Activation('softmax'))(o)
##

In [12]:
model = Model(img_input, o)

#### 查看模型架構

In [13]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 512, 512, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 518, 518, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 256, 256, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (None, 258, 258, 64) 0           conv1_conv[0][0]                 
____________________________________________________________________________________________

#### 載入模型參數(optional)

In [ ]:
# model.load_weights('DATA_0811_2\WW600WL100\\Model_0905_V2\\ResNet152V2-Unet_100.h5')

### 編譯模型

In [14]:
from keras.optimizers import Adam, Adadelta
model.compile(loss = 'categorical_crossentropy',
              optimizer = Adadelta(lr = 1.0),
              metrics = ['accuracy'])

In [15]:
import keras.backend as K
print(K.eval(model.optimizer.lr)) # 確保是1.0

1.0


### 訓練模型

In [16]:
st = time.time()
print(f'########## 即將用該資料集進行訓練：{dataset_dir} ##########')

if start_from_latest_checkpoints:
    try:
        model.load_weights(
            os.path.join(
                checkpoints_path, 
                model_name + '_' + str(max([int(i.split('_')[-1][:-3]) for i in os.listdir(checkpoints_path)])) + '.h5'
            )
        )
    except:
        print('Model weights not found!')

if not os.path.exists(checkpoints_path):
    os.makedirs(checkpoints_path)
    print('-----建立新資料夾：' + checkpoints_path + '-----') 


model.fit_generator(
    train_gen,
    steps_per_epoch = 512, 
    #steps_per_epoch = len(x_train) // BATCH_SIZE,
    epochs = EPOCHS,
    verbose = 1,
    callbacks=callbacks
)

ed = time.time()
spend_time = ed - st
print('花費時間(秒)：' + str(spend_time))

########## 即將用該資料集進行訓練：DATA_0811_2\WW600WL100 ##########
-----建立新資料夾：DATA_0811_2\WW600WL100\Model_0905_V2-----
Epoch 1/100
512/512 [==============================] - 303s 591ms/step - loss: 0.1196 - accuracy: 0.9718
Epoch 2/100
512/512 [==============================] - 258s 505ms/step - loss: 0.0493 - accuracy: 0.9885
Epoch 3/100
512/512 [==============================] - 260s 508ms/step - loss: 0.0490 - accuracy: 0.9884
Epoch 4/100
512/512 [==============================] - 259s 507ms/step - loss: 0.0322 - accuracy: 0.9911
Epoch 5/100
512/512 [==============================] - 259s 506ms/step - loss: 0.0153 - accuracy: 0.9953
Epoch 6/100
512/512 [==============================] - 259s 506ms/step - loss: 0.0073 - accuracy: 0.9976
Epoch 7/100
512/512 [==============================] - 259s 506ms/step - loss: 0.0086 - accuracy: 0.9974
Epoch 8/100
512/512 [==============================] - 260s 508ms/step - loss: 0.0087 - accuracy: 0.9972
Epoch 9/100
512/512 [============================

512/512 [==============================] - 259s 506ms/step - loss: 0.0010 - accuracy: 0.9996
Epoch 78/100
512/512 [==============================] - 260s 507ms/step - loss: 8.8580e-04 - accuracy: 0.9996
Epoch 79/100
512/512 [==============================] - 259s 506ms/step - loss: 0.0011 - accuracy: 0.9996
Epoch 80/100
512/512 [==============================] - 259s 507ms/step - loss: 9.5032e-04 - accuracy: 0.9996
Epoch 81/100
512/512 [==============================] - 259s 507ms/step - loss: 9.8110e-04 - accuracy: 0.9996
Epoch 82/100
512/512 [==============================] - 259s 506ms/step - loss: 9.6342e-04 - accuracy: 0.9996
Epoch 83/100
512/512 [==============================] - 259s 506ms/step - loss: 9.7359e-04 - accuracy: 0.9996
Epoch 84/100
512/512 [==============================] - 259s 507ms/step - loss: 9.6664e-04 - accuracy: 0.9996
Epoch 85/100
512/512 [==============================] - 259s 506ms/step - loss: 8.7170e-04 - accuracy: 0.9997
Epoch 86/100
512/512 [=========

### 模型預測結果(測試集)

In [17]:
from evaluate import *
preds = predict_from_folder(model, 'DATA_0811_2\\WW600WL100\\test\\images', INPUT_SIZE, 2)

100%|████████████████████████████████████████████████████████████████████████████| 12656/12656 [15:55<00:00, 13.25it/s]


In [18]:
st = time.time()

result = evaluate_model(
    image_dir = os.path.join(dataset_dir, 'test', 'images'), 
    label_dir = os.path.join(dataset_dir, 'test', 'annotations_tumor as kidney'), 
    checkpoints_path = None,
    calculate_predicting_indicators = True,
    output_predicted_result = False, 
    segment_out_predicted_region_from_original_images = False, 
    roi_description = 'tumor as kidney', 
    preds = preds)

print('訓練集預測結果：')
print(f'average Dice score per case of kidney: {result[0]: .4f}')
print(f'average recall of kidney: {result[1]: .4f}')
print(f'average precision of kidney: {result[2]: .4f}')
print(f'global dice score of kidney: {result[3]: .4f}')
print('')
print('混淆矩陣：')
print(f'True Positive: {result[-4]}')
print(f'False Positive: {result[-3]}')
print(f'False Negative: {result[-2]}')
print(f'True Negative: {result[-1]}')

ed = time.time()
spend_time = ed - st
print('花費時間(秒)：' + str(spend_time))

----------開始計算各項預測指標----------
目前進度：第500張照片
目前進度：第1000張照片
目前進度：第1500張照片
目前進度：第2000張照片
目前進度：第2500張照片
目前進度：第3000張照片
目前進度：第3500張照片
目前進度：第4000張照片
目前進度：第4500張照片
目前進度：第5000張照片
目前進度：第5500張照片
目前進度：第6000張照片
目前進度：第6500張照片
目前進度：第7000張照片
目前進度：第7500張照片
目前進度：第8000張照片
目前進度：第8500張照片
目前進度：第9000張照片
目前進度：第9500張照片
目前進度：第10000張照片
目前進度：第10500張照片
目前進度：第11000張照片
目前進度：第11500張照片
目前進度：第12000張照片
目前進度：第12500張照片
total case number: 12656
訓練集預測結果：
average Dice score per case of kidney:  0.9596
average recall of kidney:  0.9425
average precision of kidney:  0.9789
global dice score of kidney:  0.9526

混淆矩陣：
True Positive: 4021
False Positive: 72
False Negative: 98
True Negative: 8465
花費時間(秒)：961.4984998703003


In [19]:
f = open("DATA_0811_2\\patient indices of testing set - KiTS.txt", "r")
test_patient_idx = f.read().splitlines()
f.close()

dice_score_list = result[5]
print('測試集各病患的 Dice score:')
for idx, i in enumerate(dice_score_list):
    print(f'case{test_patient_idx[idx]}: {i: .4f}')

測試集各病患的 Dice score:
case150:  0.9766
case151:  0.8081
case152:  0.9799
case153:  0.9788
case154:  0.9587
case155:  0.9593
case156:  0.9716
case157:  0.9700
case158:  0.9744
case159:  0.9714
case160:  0.9608
case161:  0.9533
case162:  0.9626
case163:  0.9720
case164:  0.9669
case165:  0.9671
case166:  0.9706
case167:  0.9833
case168:  0.9630
case169:  0.9745
case170:  0.9533
case171:  0.9761
case172:  0.9522
case173:  0.9634
case174:  0.9740
case175:  0.9623
case176:  0.9614
case177:  0.9398
case178:  0.8938
case179:  0.9743
case181:  0.9677
case182:  0.9635
case183:  0.9648
case184:  0.9244
case185:  0.9779
case186:  0.9759
case187:  0.9299
case188:  0.9593
case189:  0.9076
case190:  0.9597
case191:  0.9729
case192:  0.9475
case193:  0.9787
case194:  0.9469
case195:  0.9689
case196:  0.9811
case197:  0.9450
case198:  0.9579
case199:  0.9543
case200:  0.9666
case201:  0.9801
case202:  0.9717
case204:  0.9781
case205:  0.9541
case206:  0.9490
case207:  0.9744
case208:  0.9464
case209:  0

In [ ]:
# st = time.time()

# from evaluate import *
# _ = evaluate_model(
#     image_dir = os.path.join(dataset_dir, 'test', 'images'), 
#     label_dir = os.path.join(dataset_dir, 'test', 'annotations_tumor as kidney'), 
#     checkpoints_path = None,
#     calculate_predicting_indicators = False,
#     output_predicted_result = True, 
#     segment_out_predicted_region_from_original_images = True, 
#     roi_description = 'tumor as kidney', 
#     preds = preds
# )

# ed = time.time()
# spend_time = ed - st
# print('花費時間(秒)：' + str(spend_time))

In [ ]:
# show_result(
#     target_dataset_base_dir = 'DATA_0811_2\\WW600WL100\\test',
#     result_num = 10,
#     roi_description = 'tumor as kidney', 
#     roi_name_chinese = '腎臟',
#     show_predicted_result = True,
#     show_segmentation_result = True, 
#     image_scale = 4)